<a href="https://colab.research.google.com/github/uliinterco/botBinance/blob/main/B1/botBinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-binance

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# needed for the binance API and websockets
# necesaria la API de binance y websockets
from binance.client import Client

# used for dates
# usar las fechas
from datetime import datetime, timedelta
import time

# used to repeatedly execute the code
# utiliza para ejecutar repetidamente el código
from itertools import count

# used to store trades and sell assets
import json

In [ ]:
# Define your API keys below in order for the toggle to work
TESTNET = True

# Get binance key and secret for TEST and MAINNET
# The keys below are pulled from environment variables using os.getenv

# Obtenga la clave binance y el secreto para TEST y MAINNET
# Las claves a continuación se extraen de las variables de entorno usando os.getenv
API_KEY = "3WSepunyDW5y8KWSsuKL0QJXQFtCDd1afchkNMICBwzCRV86r0Ai5uMeGEkNKM8R"
API_SECRET = "sXyZV3pKoz3JTiJaGLYOtP38ZEDEyUewlbKatoXs7P70tndnBvUKd73omGAsCOlO"

# Authenticate with the client
# Autenticar con el cliente
if TESTNET:
    client = Client(API_KEY, API_SECRET)

    # The API URL needs to be manually changed in the library to work on the TESTNET
    # The API URL needs to be manually changed in the library to work on the TESTNE
    client.API_URL = 'https://testnet.binance.vision/api'

else:
    client = Client(API_KEY, API_SECRET)



In [ ]:
# select what to pair the coins to and pull all coins paied with PAIR_WITH
#PAIR_WITH define la moneda (o moneda fiduciaria) con la que se empareja cada criptografía. 
# Solo he probado este USDT, ya que la mayoría de las monedas están emparejadas con él.
PAIR_WITH = 'USDT'

# Define the size of each trade, by default in USDT
#QUANTITY representa el tamaño de su operación, por defecto en USDT. 
#Tenga mucho cuidado con la CANTIDAD si cambia PAIR_WITH a BNB, por ejemplo.
QUANTITY = 100

# List of pairs to exlcude
# by default we're excluding the most popular fiat pairs
# and some margin keywords, as we're only working on the SPOT account

#FIATS es una lista de monedas fiduciarias y símbolos de margen que estoy excluyendo. 
# Todo lo que se agregue aquí se excluirá de la salida de monedas y no se intercambiará.
FIATS = ['EURUSDT', 'GBPUSDT', 'JPYUSDT', 'USDUSDT', 'DOWN', 'UP']

# the amount of time in MINUTES to calculate the differnce from the current price

#TIME_DIFFERENCE por defecto estamos comprobando la diferencia de precio de cada moneda en Binance en los últimos 5 minutos, puedes cambiar este valor para obtener diferentes resultados. 
#Esto también determina la frecuencia con la que se ejecuta cada iteración de código.
TIME_DIFFERENCE = 5

# the difference in % between the first and second checks for the price, by default set at 10 minutes apart.
CHANGE_IN_PRICE = 3
#CHANGE_IN_PRICE el umbral en el que el bot decidirá comprar una moneda. Por defecto, si una moneda se ha movido más del 3% en los últimos 5 minutos, consideramos que es una fuerte señal de compra.

# define in % when to sell a coin that's not making a profit
STOP_LOSS = 3
# define in % when to take profit on a profitable coin
TAKE_PROFIT = 6
#STOP LOSS y TAKE PROFIT definen en % cómo vender las monedas que compró el algoritmo Binance.

In [ ]:
# coins that bought by the bot since its start
coins_bought = {}
# monedas que compró el bot desde su inicio

# path to the saved coins_bought file
# ruta al archivo guardado de monedas_comprado 
coins_bought_file_path = 'coins_bought.json'

# use separate files for testnet and live
# usar archivos separados para testnet y live
if TESTNET:
    coins_bought_file_path = 'testnet_' + coins_bought_file_path

# if saved coins_bought json file exists then load it
# si existe el archivo json de monedas_comprado guardado, entonces cárguelo
if os.path.isfile(coins_bought_file_path):
    with open(coins_bought_file_path) as file:
        coins_bought = json.load(file)

In [ ]:
#The get_price() function will the return the price of each coin that meets our criteria.
#La función get_price() devolverá el precio de cada moneda que cumpla con nuestros criterios.
def get_price():
    '''Return the current price for all coins on binance'''

    initial_price = {}
    prices = client.get_all_tickers()

    for coin in prices:

        # only Return USDT pairs and exlcude margin symbols like BTCDOWNUSDT
        if PAIR_WITH in coin['symbol'] and all(item not in coin['symbol'] for item in FIATS):
            initial_price[coin['symbol']] = { 'price': coin['price'], 'time': datetime.now()}

    return initial_price

In [ ]:
#TIME_DIFFERENCE variable and will return any coin that has moved by more than the CHANGE_IN_PRICE – by default 3%.
#TIME_DIFFERENCE variable y devolverá cualquier moneda que se haya movido más que CHANGE_IN_PRICE, por defecto 3%.
def wait_for_price():
    '''calls the initial price and ensures the correct amount of time has passed
    before reading the current price again'''

    volatile_coins = {}
    initial_price = get_price()

    while initial_price['BNBUSDT']['time'] > datetime.now() - timedelta(minutes=TIME_DIFFERENCE):
        print(f'not enough time has passed yet...')

        # let's wait here until the time passess...
        time.sleep(60*TIME_DIFFERENCE)

    else:
        last_price = get_price()

        # calculate the difference between the first and last price reads
        # calcular la diferencia entre la primera y la última lectura de precio
        for coin in initial_price:
            threshold_check = (float(initial_price[coin]['price']) - float(last_price[coin]['price'])) / float(last_price[coin]['price']) * 100

            # each coin with higher gains than our CHANGE_IN_PRICE is added to the volatile_coins dict
            # cada moneda con mayores ganancias que CHANGE_IN_PRICE se agrega al dict volatile_coins
            if threshold_check > CHANGE_IN_PRICE:
                volatile_coins[coin] = threshold_check
                volatile_coins[coin] = round(volatile_coins[coin], 3)

                print(f'{coin} has gained {volatile_coins[coin]}% in the last {TIME_DIFFERENCE} minutes, calculating volume in {PAIR_WITH}')

        if len(volatile_coins) < 1:
                print(f'No coins moved more than {CHANGE_IN_PRICE}% in the last {TIME_DIFFERENCE} minute(s)')

        return volatile_coins, len(volatile_coins), last_price

In [ ]:
#The next step is to convert our QUANTITY of 100USDT (default) into the respective quantity for each coin that we’re about to buy. 
#El siguiente paso es convertir nuestra CANTIDAD de 100USDT (predeterminada) en la cantidad respectiva de cada moneda que estamos a punto de comprar.
def convert_volume():
    '''Converts the volume given in QUANTITY from USDT to the each coin's volume'''
#Because Binance is a bit particular with the format of the volume, our trading bot needs to know the step size for each coin.
#Debido a que Binance es un poco particular con el formato del volumen, nuestro robot comercial necesita saber el tamaño de paso para cada moneda.

    volatile_coins, number_of_coins, last_price = wait_for_price()
    lot_size = {}
    volume = {}

    for coin in volatile_coins:

        # Find the correct step size for each coin -------------->Encuentre el tamaño de paso correcto para cada moneda
        # max accuracy for BTC for example is 6 decimal points--->la precisión máxima para BTC, por ejemplo, es de 6 puntos decimales
        # while XRP is only 1 ----------------------------------->mientras que XRP es solo 1
        try:
            info = client.get_symbol_info(coin)
            step_size = info['filters'][2]['stepSize']
            lot_size[coin] = step_size.index('1') - 1

            if lot_size[coin] < 0:
                lot_size[coin] = 0

        except:
            pass

        # calculate the volume in coin from QUANTITY in USDT (default)
        # calcular el volumen en moneda de CANTIDAD en USDT (predeterminado)
        volume[coin] = float(QUANTITY / float(last_price[coin]['price']))

        # define the volume with the correct step size
        # definir el volumen con el tamaño de paso correcto
        if coin not in lot_size:
            volume[coin] = float('{:.1f}'.format(volume[coin]))

        else:
            volume[coin] = float('{:.{}f}'.format(volume[coin], lot_size[coin]))

    return volume, last_price

In [ ]:
def trade():
    '''Place Buy market orders for each volatile coin found'''

    volume, last_price = convert_volume()
    orders = {}

    for coin in volume:

        # only buy if the there are no active trades on the coin
        # solo compra si no hay intercambios activos en la moneda
        if coin not in coins_bought or coins_bought[coin] == None:
            print(f' preparing to buy {volume[coin]} {coin}')

            if TESTNET :
                # create test order before pushing an actual order
                # crear un pedido de prueba antes de enviar un pedido real
                test_order = client.create_test_order(symbol=coin, side='BUY', type='MARKET', quantity=volume[coin])

            # try to create a real order if the test orders did not raise an exception
            # intenta crear un pedido real si los pedidos de prueba no generaron una excepción
            try:
                buy_limit = client.create_order(
                    symbol=coin,
                    side='BUY',
                    type='MARKET',
                    quantity=volume[coin]
                )

            # error handling here in case position cannot be placed
            except Exception as e:
                print(e)

            # run the else block if the position has been placed and return order info
            # ejecutar el bloque else si se ha colocado la posición y devolver la información del pedido
            else:
                orders[coin] = client.get_all_orders(symbol=coin, limit=1)
        else:
            print(f'Signal detected, but there is already an active trade on {coin}')

    return orders, last_price, volume

In [ ]:
#The next step is to update our portfolio by saving the details of each trade into the json file that we’re checking for at the beginning of each iteration.
#El siguiente paso es actualizar nuestra cartera guardando los detalles de cada operación en el archivo json que buscamos al comienzo de cada iteración.
def update_porfolio(orders, last_price, volume):
    '''add every coin bought to our portfolio for tracking/selling later'''

    for coin in orders:
        coins_bought[coin] = {
            'symbol': orders[coin][0]['symbol'],
            'orderid': orders[coin][0]['orderId'],
            'timestamp': orders[coin][0]['time'],
            'bought_at': last_price[coin]['price'],
            'volume': volume[coin]
            }

        # save the coins in a json file in the same directory
        # guardar las monedas en un archivo json en el mismo directorio
        with open(coins_bought_file_path, 'w') as file:
            json.dump(coins_bought, file, indent=4)


In [ ]:
#checks if any of the coins we own in our bot portfolio should be sold due to SL or TP being reached.
#comprueba si alguna de las monedas que poseemos en nuestra cartera de bots debe venderse debido a que se alcanzó SL o TP.
def sell_coins():
    '''sell coins that have reached the STOP LOSS or TAKE PROFIT thershold'''

    last_price = get_price()

    for coin in coins_bought:
        # define stop loss and take profit
        TP = float(coins_bought[coin]['bought_at']) + (float(coins_bought[coin]['bought_at']) * TAKE_PROFIT) / 100
        SL = float(coins_bought[coin]['bought_at']) - (float(coins_bought[coin]['bought_at']) * STOP_LOSS) / 100

        # check that the price is above the take profit or below the stop loss
        # check that the price is above the take profit or below the stop loss
        if float(last_price[coin]['price']) > TP or float(last_price[coin]['price']) < SL:
            print(f"TP or SL reached, selling {coins_bought[coin]['volume']} {coin}...")

            if TESTNET :
                # create test order before pushing an actual order
                # crear un pedido de prueba antes de enviar un pedido real
                test_order = client.create_test_order(symbol=coin, side='SELL', type='MARKET', quantity=coins_bought[coin]['volume'])

            # try to create a real order if the test orders did not raise an exception
            # intentar crear un pedido real si los pedidos de prueba no generaron una excepción
            try:
                sell_coins_limit = client.create_order(
                    symbol=coin,
                    side='SELL',
                    type='MARKET',
                    quantity=coins_bought[coin]['volume']
                )

            # error handling here in case position cannot be placed
            except Exception as e:
                print(e)

            # run the else block if the position has been placed and update the coins bought json file
            # ejecutar el bloque else si se ha colocado la posición y actualizar el archivo json de monedas compradas
            else:
                coins_bought[coin] = None
                with open(coins_bought_file_path, 'w') as file:
                    json.dump(coins_bought, file, indent=4)
        else:
            print(f'TP or SL not yet reached, not selling {coin} for now...')

print('Press Ctrl-Q to stop the script')
for i in count():
        orders, last_price, volume = trade()
        update_porfolio(orders, last_price, volume)
        sell_coins()

Press Ctrl-Q to stop the script
not enough time has passed yet...
No coins moved more than 3% in the last 5 minute(s)
not enough time has passed yet...
No coins moved more than 3% in the last 5 minute(s)
not enough time has passed yet...
No coins moved more than 3% in the last 5 minute(s)
not enough time has passed yet...
